In [1]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd

In [2]:
res = []

for i in range(381):
    r = requests.get('https://www.olx.ba/pretraga?trazilica=+golf&kategorija=18&stranica='+ str(i))
    soup = BeautifulSoup(r.text, 'lxml')

    all_items = soup.select('div#rezultatipretrage div.listitem.artikal')

    for item in all_items:

        if not item.find('p', class_='na'):
            continue

        naziv = item.find('p', class_='na').text
        
        link = item.a['href']
        
        lokacija = item.find('div', class_='lokacijadiv').text.strip()
        
        godiste = item.find('span', class_='desnopolje').text if item.find('span', class_='desnopolje') else float('NaN')
        
        try:
            gorivo = item.find_all('p', class_='polje')[1].find('span', class_='desnopolje').text
        except IndexError:
            gorivo = 'NaN'

        if item.find('div', class_='cijena').span.text == 'PO DOGOVORU':
            cijena = 'PO DOGOVORU'
        else:
            cijena = item.find('div', class_='cijena').span.contents[-1][:-2].strip()
            
            if ',' in cijena:
                continue
                
            cijena = float(cijena.replace('.',''))

        stanje = item.find('div', class_='stanje k').text.strip() if item.find('div', class_='stanje k') else float('NaN')
        datum = item.find('div', class_='kada').text

        res.append((naziv, stanje, godiste, gorivo, cijena, lokacija, datum, link))

In [3]:
df = pd.DataFrame(res, columns=['NAZIV_ARTIKLA', 'STANJE', 'GODISTE', 'GORIVO', 'CIJENA', 'LOKACIJA', 'DATUM', 'LINK'])

In [4]:
df.head()

,NAZIV_ARTIKLA,STANJE,GODISTE,GORIVO,CIJENA,LOKACIJA,DATUM,LINK
0,Golf 4 1.9 tdi 66kw reg 3/2019,KORIŠTENO,2001,Dizel,5350,Breza,Prije 3 dana,https://www.olx.ba/artikal/30499130/golf-4-1-9...
1,Volkswagen Golf V 5 1.9 TDI 6 brzina,KORIŠTENO,2004,Dizel,8350,Pale,11.05.2018 u 12:33,https://www.olx.ba/artikal/29642465/volkswagen...
2,Volkswagen Golf 1.9 tdi 77kw,KORIŠTENO,2004,Dizel,7900,Visoko,Prije 2 dana,https://www.olx.ba/artikal/30515099/volkswagen...
3,GOLF 7 VII DSG 1.6TDI NAVIGACIJA ACC KOMANDE FULL,KORIŠTENO,2013,Dizel,24500,Sarajevo - Centar,Prije 4 dana,https://www.olx.ba/artikal/30494439/golf-7-vii...
4,GOLF 5 2.0 TDI 103kw,KORIŠTENO,2004,Dizel,8650,Gradačac,"Jučer, 15:39",https://www.olx.ba/artikal/30521990/golf-5-2-0...


In [5]:
df.tail()

,NAZIV_ARTIKLA,STANJE,GODISTE,GORIVO,CIJENA,LOKACIJA,DATUM,LINK
11419,Golf 4,KORIŠTENO,1999,Dizel,4600,,24.12.2017 u 09:59,https://www.olx.ba/artikal/27603673/golf-4/
11420,Volkswagen Golf V,KORIŠTENO,2005,Dizel,9800,Sarajevo - Centar,23.12.2017 u 23:02,https://www.olx.ba/artikal/26750006/volkswagen...
11421,Volkswagen Golf,KORIŠTENO,1985,Dizel,800,Gradačac,23.12.2017 u 22:55,https://www.olx.ba/artikal/27956942/volkswagen...
11422,Golf 2 dizel,KORIŠTENO,1988,Dizel,1300,Zenica,23.12.2017 u 20:51,https://www.olx.ba/artikal/27955720/golf-2-dizel/
11423,Golf 2,KORIŠTENO,1986,Dizel,1300,Nevesinje,23.12.2017 u 17:37,https://www.olx.ba/artikal/28759448/golf-2/


In [6]:
df.shape

(11424, 8)

In [8]:
df.to_csv('olx_golf_data.csv', index=False)